In [10]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from pydantic import BaseModel, Field
import operator

load_dotenv()
model = ChatOpenAI(model='gpt-4o-mini')

In [3]:
class evaluationSchema(BaseModel):
  feedback: str = Field(description="Detailed Feedback for the essay.")
  score: int = Field(description="score out of 10", ge=0, le=10)

In [4]:
structured_model = model.with_structured_output(evaluationSchema)


In [23]:
essay1 = """Tourism in India: A Tapestry of Heritage and Progress
India is often described not just as a country, but as a continent of experiences. From the snow-capped peaks of the Himalayas to the tropical backwaters of Kerala, tourism in India is a multi-faceted industry that serves as a bridge between the nation’s glorious past and its ambitious future. As of 2026, the sector has evolved from traditional sightseeing into a sophisticated engine of economic growth, cultural diplomacy, and sustainable development.

The Pillars of Indian Tourism
The diversity of India’s landscape and culture allows for several distinct "niches" that cater to global and domestic travelers alike:

Cultural and Heritage Tourism: This remains the backbone of the industry. Iconic monuments like the Taj Mahal, the Red Fort, and the ancient temples of Hampi or Madurai continue to draw millions. Recently, the government has focused on developing "Archaeological Experiential Sites" at locations like Lothal and Sarnath, blending conservation with immersive storytelling.
+1

Spiritual and Pilgrimage Tourism: India is the spiritual heart of the world. Beyond the traditional circuits like the Char Dham or Varanasi, there is a renewed push for the Buddhist Circuit across the North-East, aiming to connect international pilgrims with the roots of Buddhist philosophy.

Medical and Wellness Tourism: Under the "Heal in India" initiative, the country has become a premier global healthcare destination. By offering world-class surgical procedures and traditional Ayurveda at a fraction of Western costs, India is attracting a new demographic of "medical value travelers."
+1

Eco and Adventure Tourism: With a growing global focus on the environment, India has introduced thematic trails, such as Turtle Trails in Odisha and Mountain Trails in Jammu & Kashmir. These initiatives encourage responsible travel that supports local biodiversity.
+1

Economic Impact and Government Initiatives
Tourism is a vital contributor to the Indian economy, accounting for approximately 6–7% of the national GDP. It is one of the largest employers in the country, supporting over 40 million jobs ranging from high-end hospitality to local artisans and guides.
+1

To reach the target of a $3 trillion tourism economy by 2047, the government has launched several strategic programs:

Union Budget 2026-27: Recent allocations focus on upskilling 10,000 tour guides in collaboration with IIMs and transforming the National Council for Hotel Management into a world-class National Institute of Hospitality.

Infrastructure Development: The Swadesh Darshan 2.0 and PRASHAD schemes are modernizing the infrastructure at 50 "challenge-mode" destinations, ensuring they meet international standards for cleanliness and connectivity.

Digital Integration: The Unified Tourism Interface—linking UPI, DigiYatra, and ONDC—has revolutionized the traveler’s journey, making payments and movement across states seamless.

Challenges and the Road Ahead
Despite its potential, the sector faces hurdles. Over-tourism in popular hill stations, infrastructure gaps in remote areas, and the need for stricter safety standards for women are ongoing concerns. Furthermore, while the "Incredible India" brand is iconic, there is a push in 2026 to overhaul it with "country-specific strategies," recognizing that a traveler from Japan has different needs and interests than one from Germany.
+1

Conclusion
Tourism in India is no longer just about the "Golden Triangle" of Delhi, Agra, and Jaipur. It is about the "Glowmads" seeking beauty rituals, the hikers exploring the Eastern Ghats, and the patients seeking healing. By balancing its ancient philosophy of "Atithi Devo Bhava" (The Guest is God) with modern technology and sustainable practices, India is well on its way to becoming a global tourism superpower."""

essay2 = """India tourism is like a big biryani of many things because many people come here for see many things like Taj Mahal which is white and very big and also old. Actually tourism is good for money of India because many people give money for hotels and autos and that is why India is growing like a rocket in the sky of world.

In India there is many weather like very hot and very cold and also rain. If you go to North there is mountain with ice but if you go to South there is water and coconut trees which is looking very green. Some people say India is very crowd but I say India is very heart because people are very helping and they say guest is like God which is a very old saying from long ago.

Also there is food which is very spicy and many foreigners like it but some get stomach problem but still they eat because taste is very yum. Many temples are there with many Gods and people ring bell and it feels very peace in the mind.

Government is also doing things like making roads and trains which is running fast like Vande Bharat which is a blue and white train and looks very modern. Tourism is very important because if no tourism then how we show our culture to the world people? So we must keep India clean and not throw plastic on the road.

Overall tourism is a very big essay topic because India is a very big country and everyone should come here at least one time in their life to see the beauty of India and eat the food and take many photos for mobile and Instagram."""

In [8]:
#Testing, the structured output working properly or not.
prompt1 = f'evaluate the language quality of the following essay and give a proper feedback. Rate the essay out of 10. here is the essay - \n {essay1}'
result = structured_model.invoke(prompt1)
print(result)

feedback='The essay titled "Tourism in India: A Tapestry of Heritage and Progress" is well-organized and informative, showcasing the various facets of Indian tourism and the measures being taken for its advancement. The language used is generally clear and articulate, with a sophisticated vocabulary that enhances the overall quality of the writing. Here are some specific points of feedback:\n\n1. **Structure and Coherence**: The essay is structured logically, with clear headings for each subsection. This aids the reader in following the narrative about Indian tourism, its growth, its various niches, and the associated challenges. Each section flows smoothly into the next, contributing to the overall coherence.\n\n2. **Content Depth**: The essay provides a thorough overview of the diverse tourism sectors in India, including cultural, spiritual, medical, wellness, eco, and adventure tourism. Additionally, it touches on the economic impact and various government initiatives, which enrich 

In [15]:
class upscState(TypedDict):
  essay: str
  language_feedback: str
  analysis_feedback: str
  clarity_feedback: str
  individual_score: Annotated[list[int], operator.add]
  summary_feedback: str
  average_score: float

In [16]:
def language_feedback(state: upscState):
  prompt = f'Evaluage the language quality of the following essay and provide a feedback. And assign a rating out of 10. Here is the essay - \n {state["essay"]}'

  result = structured_model.invoke(prompt)

  return {'language_feedback':result.feedback, 'individual_score': [result.score]}

def analysis_feedback(state: upscState):
  prompt = f'Evaluage the analysis quality of the following essay and provide a feedback. And assign a rating out of 10. Here is the essay - \n {state["essay"]}'

  result = structured_model.invoke(prompt)

  return {'analysis_feedback':result.feedback, 'individual_score': [result.score]}

def clarity_feedback(state: upscState):
  prompt = f'Evaluage the clarity of thought of the following essay and provide a feedback. And assign a rating out of 10. Here is the essay - \n {state["essay"]}'

  result = structured_model.invoke(prompt)

  return {'clarity_feedback':result.feedback, 'individual_score': [result.score]}

def final_evaluation(state: upscState):
  prompt = f'based on the following feedback create ta final summary feedback. language feedback - {state["language_feedback"]}\n analysis feedback - {state["analysis_feedback"]} \n clarity of thought feedback - {state["clarity_feedback"]}.'

  final_output = model.invoke(prompt).content

  avg_score = sum(state['individual_score']) / len(state['individual_score'])

  return {'summary_feedback':final_output, 'average_score':avg_score}


In [19]:
graph = StateGraph(upscState)

graph.add_node('language_feedback', language_feedback)
graph.add_node('analysis_feedback', analysis_feedback)
graph.add_node('clarity_feedback', clarity_feedback)
graph.add_node('summary_feedback', final_evaluation)

graph.add_edge(START, 'language_feedback')
graph.add_edge(START, 'analysis_feedback')
graph.add_edge(START, 'clarity_feedback')

graph.add_edge('language_feedback', 'summary_feedback')
graph.add_edge('analysis_feedback', 'summary_feedback')
graph.add_edge('clarity_feedback', 'summary_feedback')

graph.add_edge('summary_feedback', END)

workflow = graph.compile()

In [ ]:
initial_state = {'essay': essay2}

workflow.invoke(initial_state)



{'essay': 'India tourism is like a big biryani of many things because many people come here for see many things like Taj Mahal which is white and very big and also old. Actually tourism is good for money of India because many people give money for hotels and autos and that is why India is growing like a rocket in the sky of world.\n\nIn India there is many weather like very hot and very cold and also rain. If you go to North there is mountain with ice but if you go to South there is water and coconut trees which is looking very green. Some people say India is very crowd but I say India is very heart because people are very helping and they say guest is like God which is a very old saying from long ago.\n\nAlso there is food which is very spicy and many foreigners like it but some get stomach problem but still they eat because taste is very yum. Many temples are there with many Gods and people ring bell and it feels very peace in the mind.\n\nGovernment is also doing things like making 